In [1]:
using Clustering
using Mocha
using MultivariateStats
using JLD
using Debug
using StatsBase
using Distributions

Configuring Mocha...
 * CUDA       disabled by default
 * Native Ext disabled by default
Mocha configured, continue loading module...


DefaultBackend = Mocha.CPUBackend


In [2]:
file="dataset_config.txt"
f = open("Data"*"/"*file, "r")
for l in eachline(f)
    config = copy(l)
    break
end

In [3]:
positions, distributions = split(config, "|")
positions = map(x->parse(Float64, x), split(positions,','))
distributions = split(replace(distributions, "\n", ""), ",")
#print(positions)

LoadError: UndefVarError: config not defined

In [2]:
@debug function read_stats(fname)
  stats = jldopen(fname, "r") do file
    read(file, "statistics")
  end
  return stats
end

function number_stats(fnames, names)
  res = Any[]
  for (i, fname) in enumerate(fnames)
    for name in names[i]
      push!(res, (i, fname, name))
    end
  end
  return res
end

function list_stats(numbered_names)
  println("Listing available statistics")
  for (k, (_, fname, name)) in enumerate(numbered_names)
    println("  $k : $fname/$name")
  end
  println("Select statistics to plot using -i and specify the numbers 1-$(length(numbered_names)) seperated with ,")
end

function create_safe_files(fnames, to_tmp)
  # copy to temporary file if requested
  if to_tmp
    stats_files = [tempname() for fname in fnames]
    for (tmpfile,fname) in zip(stats_files, fnames)
      cp(fname, tmpfile)
    end
    return stats_files
  else
    return fnames
  end
end

LoadError: UndefVarError: @debug not defined

In [1]:
filenames = ["Snapshots/snapshots_2/snapshot-000000.jld", "Snapshots/snapshots_2/snapshot-001000.jld"]
stats_files = create_safe_files(filenames, true)
all_stats = map(read_stats, stats_files)
# get all unique statistic names that were logged in each files
#names = map(keys, all_stats)
# and assign a number to each
#numbered_names = number_stats(filenames, names)

LoadError: UndefVarError: create_safe_files not defined

In [29]:
file = jldopen(filenames[1])

Julia data file version 0.1.1: Snapshots/snapshots_2/snapshot-000000.jld

In [40]:
stats = read(file, "solver_state")
keys(stats)

LoadError: MethodError: `keys` has no method matching keys(::Mocha.AdamSolverStateSnapshot)

In [4]:
tic()
a = 2
time = toc()

elapsed time: 0.011958373 seconds


0.011958373

In [5]:
@everywhere function R2(predictions, y)
    SSres = sum((predictions .- y).^2)
    ymean = mean(y)
    SStot = sum((y .- ymean).^2)
    return 1 - (SSres/SStot)
end

In [9]:
y_true = [3., -0.5, 2., 7.]
y_pred = [2.5, 0.0, 2., 8,]
#R2(y_pred, y_true)
y_true = [1.,2.,3.]
y_pred = [1.,2.,3.]
#R2(y_pred, y_true)
y_true = [1,2,3]
y_pred = [3,2,1]
R2(y_pred, y_true)

-3.0

In [5]:
time

0.00027987

In [5]:
nodes = 10
num_nodes = 2
all_nodes = collect(1:nodes)
modes = ceil(nodes / num_nodes)
nodes_mean = float(all_nodes%num_nodes)
positions = [2.,10.]
positions[Int(nodes_mean[2]) + 1]

2.0

In [6]:
@everywhere generate_inputs(mean, var, data_size) = transpose(rand(MvNormal(mean, var), data_size))

@everywhere f10(x) = 10.*sin(pi.*x[:,1].*x[:,2]) + 
                20(x[:,3]-0.5).^2 + 10.*x[:,4] + 5.*x[:,5] + 
                10.*sin(pi.*x[:,6].*x[:,7]) + 20(x[:,8]-0.5).^2 + 10.*x[:,9] + 5.*x[:,10] + rand(Normal(0,0.1),size(x[:,1])[1])


In [7]:
data_size = 1000
node_id = 1
dim = 10

# introspecting the output_function, we want to know how many parameters it has

mod_node = float(node_id % num_nodes)
#dim = out_fn_table.max_args

mean_vector = Array{Float32}([2.*m for m in fill(mod_node, dim)])
var_vector = eye(dim, dim)
dataset_input = generate_inputs(mean_vector, var_vector, data_size)
dataset_output = f10(dataset_input)

1000-element Array{Float64,1}:
  96.5198
  47.4839
 288.521 
 145.569 
 205.123 
 279.051 
 196.055 
 233.84  
 363.969 
 110.278 
 276.017 
 156.425 
 137.808 
   ⋮     
  43.8109
 109.312 
 169.719 
 400.734 
 148.055 
  84.922 
 201.459 
 322.889 
 241.008 
 308.054 
 101.155 
  59.9231

In [8]:
print(size(dataset_input))
print(size(dataset_output))

(1000,10)(1000,)

In [12]:
id = "3"
input_data = load("Data/data_"*id*"/input.jld", "dataset_input")
dataset1 = input_data
#id = "5"
#input_data = load("Data/data_"*id*"/input.jld", "dataset_input")
#dataset2 = input_data

1000x2 Array{Float64,2}:
 18.092   19.5131
 21.2438  18.4354
 20.3029  20.3655
 20.9166  20.7453
 20.7784  19.1501
 19.327   19.1952
 22.0112  19.9879
 21.0678  21.8934
 19.5748  21.0012
 20.4838  19.1972
 20.3545  19.6601
 20.038   20.4165
 20.5155  19.304 
  ⋮              
 19.7158  19.7166
 20.8446  20.824 
 20.5635  20.5587
 18.6909  19.4831
 19.5611  19.152 
 20.1026  19.8798
 20.8072  19.9847
 19.2487  20.9038
 19.7839  20.3304
 20.9284  21.1377
 18.2764  19.9181
 18.6732  20.0687

In [13]:
dataset = input_data
examples, attributes = try
    e, a = size(dataset)
catch
    e, a = (size(dataset)[1], 1)
end
data = tuple(dataset[:,1])
for attribute in 2:attributes
    data = tuple(data..., dataset[:,attribute])
end

# Test training classifier

In [14]:
@everywhere function shufflerows!{T<:Real}(a::AbstractArray{T})
    for i = size(a, 1):-1:2
        j = rand(1:i)
        a[i,:], a[j,:] = a[j,:], a[i,:]
    end
    return a
end

In [18]:
id = string(3)
datainput = load("Data/data_"*id*"/input.jld", "dataset_input")
datalabels = load("Data/data_"*id*"/output.jld", "dataset_output")

dataset=[datainput datalabels]


1000x3 Array{Float64,2}:
 18.092   19.5131  -0.0409736 
 21.2438  18.4354  -0.0551551 
 20.3029  20.3655   0.123257  
 20.9166  20.7453  -0.111415  
 20.7784  19.1501  -0.112851  
 19.327   19.1952  -0.0686609 
 22.0112  19.9879   0.0690002 
 21.0678  21.8934   0.0646539 
 19.5748  21.0012  -0.0931039 
 20.4838  19.1972  -0.0927132 
 20.3545  19.6601  -0.0336073 
 20.038   20.4165   0.136934  
 20.5155  19.304    0.00807034
  ⋮                           
 19.7158  19.7166   0.0590312 
 20.8446  20.824   -0.046732  
 20.5635  20.5587  -0.114331  
 18.6909  19.4831   0.0670901 
 19.5611  19.152    0.0287877 
 20.1026  19.8798  -0.120648  
 20.8072  19.9847  -0.14126   
 19.2487  20.9038  -0.0178737 
 19.7839  20.3304   0.0456328 
 20.9284  21.1377   0.173695  
 18.2764  19.9181   0.0232349 
 18.6732  20.0687  -0.0632176 

In [19]:
(datainput .- mean(datainput, 1)) ./ var(datainput, 1)

1000x2 Array{Float64,2}:
 -2.06552    -0.49522   
  1.30591    -1.60276   
  0.299427    0.380698  
  0.955845    0.770965  
  0.808033   -0.86833   
 -0.744503   -0.821969  
  2.12676    -0.00738603
  1.11759     1.95081   
 -0.479377    1.03398   
  0.49292    -0.819895  
  0.354637   -0.344176  
  0.0161273   0.433077  
  0.526793   -0.710129  
  ⋮                     
 -0.328548   -0.286089  
  0.878822    0.851914  
  0.578217    0.579278  
 -1.42489    -0.526093  
 -0.49407    -0.86632   
  0.0851396  -0.11839   
  0.83881    -0.0106057 
 -0.828196    0.933892  
 -0.255749    0.344616  
  0.968485    1.17424   
 -1.86829    -0.0790969 
 -1.44382     0.0756735 

In [24]:
stats = [mean(datainput, 1); var(datainput, 1)]
if !isdir("Data/stats_"*id)
    mkdir("Data/stats_"*id)
end
save("Data/stats_"*id*"/stats_train.jld", "stats_train", stats)


In [22]:
means = stats[1,:]
vars = stats[2,:]
print(means)
print(vars)

[20.022969867970673 19.995040160374355][0.9348724484597412 0.973102539512446]

In [27]:
stats = load("Data/stats_"*id*"/stats_train.jld", "stats_train")

2x2 Array{Float64,2}:
 0.0224807  0.0391242
 0.942183   1.02276  

In [25]:
@everywhere function compute_stats_train(datainput)
    id = string(myid())
    stats = [mean(datainput, 1); var(datainput, 1)]
    if !isdir("Data/stats_"*id)
        mkdir("Data/stats_"*id)
    end
    save("Data/stats_"*id*"/stats_train.jld", "stats_train", stats)
    return stats
end

@everywhere function read_stats_train()
    id = string(myid()) 
    stats = load("Data/stats_"*id*"/stats_train.jld", "stats_train")
    return stats
end

@everywhere function standarize_dataset(datainput, stats)
    means = stats[1,:]
    vars = stats[2,:]
    datainput_standard = (datainput .- means) ./ vars
    return datainput_standard
end

In [26]:
stats = compute_stats_train(datainput)


2x2 Array{Float64,2}:
 20.023     19.995   
  0.934872   0.973103

In [27]:
stats = read_stats_train()

2x2 Array{Float64,2}:
 20.023     19.995   
  0.934872   0.973103

In [28]:
datainput_st = standarize_dataset(datainput, stats)

1000x2 Array{Float64,2}:
 -2.06552    -0.49522   
  1.30591    -1.60276   
  0.299427    0.380698  
  0.955845    0.770965  
  0.808033   -0.86833   
 -0.744503   -0.821969  
  2.12676    -0.00738603
  1.11759     1.95081   
 -0.479377    1.03398   
  0.49292    -0.819895  
  0.354637   -0.344176  
  0.0161273   0.433077  
  0.526793   -0.710129  
  ⋮                     
 -0.328548   -0.286089  
  0.878822    0.851914  
  0.578217    0.579278  
 -1.42489    -0.526093  
 -0.49407    -0.86632   
  0.0851396  -0.11839   
  0.83881    -0.0106057 
 -0.828196    0.933892  
 -0.255749    0.344616  
  0.968485    1.17424   
 -1.86829    -0.0790969 
 -1.44382     0.0756735 

In [29]:
mean(datainput_st, 1)

1x2 Array{Float64,2}:
 -7.27129e-15  -2.82602e-15

In [30]:
var(datainput_st, 1)

1x2 Array{Float64,2}:
 1.06966  1.02764

In [31]:

#datainput_train=sub(datainput,1:round(Int,(*(0.8,examples))),1:atributtes)
#datalabels_train=sub(datalabels,1:round(Int,(*(0.8,examples))),1)

#datainput_val=sub(datainput,+(round(Int,(*(0.8,examples))),1):examples,1:atributtes)
#exa,at=size(datainput_val)
#datalabels_val=sub(datalabels,+(round(Int,(*(0.8,examples))),1):examples,1)
maximo=Inf
global neurona=0
hidden_neurons = 20
neurona = hidden_neurons

dataset2=shufflerows!(dataset)
examples, atributes = size(dataset2)
datainput2=dataset2[:,1:atributes-1]
datalabels2=dataset2[:,atributes]
datainput_train=sub(datainput2,1:round(Int,(*(0.8,examples))),1:atributes-1)
datalabels_train=sub(datalabels2,1:round(Int,(*(0.8,examples))),1)

datainput_val=sub(datainput2,+(round(Int,(*(0.8,examples))),1):examples,1:atributes-1)
exa,at=size(datainput_val)
datalabels_val=sub(datalabels2,+(round(Int,(*(0.8,examples))),1):examples,1)

backend = CPUBackend()
init(backend)

data_train_layer = MemoryDataLayer(name="data", data=Array[transpose(datainput_train), transpose(datalabels_train)], batch_size=100)
data_val_layer = MemoryDataLayer(name="data", data=Array[transpose(datainput_val), transpose(datalabels_val)], batch_size=exa)

ip_layer = InnerProductLayer(name="ip", output_dim=hidden_neurons, bottoms=[:data], tops=[:ip], neuron=Neurons.ReLU())
ip2_layer = InnerProductLayer(name="ip2", output_dim=hidden_neurons, bottoms=[:ip], tops=[:ip2], neuron=Neurons.ReLU())
aggregator = InnerProductLayer(name="aggregator", output_dim=1, tops=[:aggregator], bottoms=[:ip2] )
layer_loss = SquareLossLayer(name="loss", bottoms=[:aggregator, :label])

common_layers = [ip_layer, ip2_layer, aggregator]

net = Net("MLP", backend, [data_train_layer;common_layers;layer_loss])

method = Adam() # adam
params = make_solver_parameters(method, max_iter=10000)
solver = Solver(method, params)

#setup_coffee_lounge(solver, save_into="Snapshots/snapshots_"*id*"/statistics.jld", every_n_iter=1000)
# report training progress every 1000 iterations
add_coffee_break(solver, TrainingSummary(), every_n_iter=1000)

val_net = Net("MLP", backend, [data_val_layer;common_layers;layer_loss])
add_coffee_break(solver, ValidationPerformance(val_net), every_n_iter=1000)  

add_coffee_break(solver, Snapshot("Snapshots/snapshots_"*id), every_n_iter=1000) 
solve(solver, net)
Mocha.dump_statistics(solver.coffee_lounge, get_layer_state(net, "loss"), true)
#destroy(net)
#destroy(val_net)
#shutdown(backend)

09-Oct 20:06:55:INFO:root:Constructing net MLP on Mocha.CPUBackend...
09-Oct 20:06:55:INFO:root:Topological sorting 5 layers...
09-Oct 20:06:55:INFO:root:Setup layers...
09-Oct 20:06:55:INFO:root:Network constructed!
09-Oct 20:06:55:INFO:root:Constructing net MLP on Mocha.CPUBackend...
09-Oct 20:06:55:INFO:root:Topological sorting 5 layers...
09-Oct 20:06:55:INFO:root:Setup layers...
09-Oct 20:06:56:DEBUG:root:InnerProductLayer(ip): sharing weights and bias
09-Oct 20:06:56:DEBUG:root:InnerProductLayer(ip2): sharing weights and bias
09-Oct 20:06:56:DEBUG:root:InnerProductLayer(aggregator): sharing weights and bias
09-Oct 20:06:56:INFO:root:Network constructed!
09-Oct 20:06:56:DEBUG:root:#DEBUG Checking network topology for back-propagation
09-Oct 20:06:57:DEBUG:root:Init network MLP
09-Oct 20:06:57:DEBUG:root:Init parameter weight for layer ip
09-Oct 20:06:57:DEBUG:root:Init parameter bias for layer ip
09-Oct 20:06:57:DEBUG:root:Init parameter weight for layer ip2
09-Oct 20:06:57:DEBUG:

In [38]:
@everywhere function evaluate_test_data_with_local_models(batch_size)
    output_predictions = zeros(batch_size)
    id = string(2)
    datainput = load("Data/data_"*id*"/input_test.jld", "dataset_input")
    dataoutput = load("Data/data_"*id*"/output_test.jld", "dataset_output")

    backend = CPUBackend()
    init(backend)

    neuro = string(20)
    neuron_number = parse(Int,neuro)
    data_layer = MemoryDataLayer(name="data", data=Array[transpose(datainput), transpose(dataoutput)], batch_size=batch_size)
    ip_layer = InnerProductLayer(name="ip", output_dim=neuron_number, bottoms=[:data], tops=[:ip], neuron=Neurons.ReLU())
    ip2_layer = InnerProductLayer(name="ip2", output_dim=neuron_number, bottoms=[:ip], tops=[:ip2], neuron=Neurons.ReLU())
    aggregator = InnerProductLayer(name="aggregator", output_dim=1, tops=[:aggregator], bottoms=[:ip2] )

    common_layers = [ip_layer, ip2_layer, aggregator]
    net = Net("MLP", backend, [data_layer; common_layers])
    data = "Snapshots/snapshots_"*id*"/snapshot-010000.jld"
    load_snapshot(net, data)
    forward(net)
    output_predictions[:] = transpose(net.output_blobs[:aggregator].data)
    destroy(net)
    shutdown(backend)

    return output_predictions
end

In [39]:
@everywhere function evaluate_test_data_with_local_models_nodes(nodes, batch_size)
    output_predictions = zeros(nodes, batch_size)
    id = string(2)
    datainput = load("Data/data_"*id*"/input_test.jld", "dataset_input")
    dataoutput = load("Data/data_"*id*"/output_test.jld", "dataset_output")

    for node in 1:nodes
        backend = CPUBackend()
        init(backend)
        neuron_number=20
        data_layer = MemoryDataLayer(name="data", data=Array[transpose(datainput), transpose(dataoutput)], batch_size=batch_size)
        ip_layer = InnerProductLayer(name="ip", output_dim=neuron_number, bottoms=[:data], tops=[:ip], neuron=Neurons.ReLU())
        ip2_layer = InnerProductLayer(name="ip2", output_dim=neuron_number, bottoms=[:ip], tops=[:ip2], neuron=Neurons.ReLU())
        aggregator = InnerProductLayer(name="aggregator", output_dim=1, tops=[:aggregator], bottoms=[:ip2] )

        common_layers = [ip_layer, ip2_layer, aggregator]
        net = Net("MLP", backend, [data_layer; common_layers])
        data = "Snapshots/snapshots_"*string(node + 1)*"/snapshot-010000.jld"
        load_snapshot(net, data)
        forward(net)
        output_predictions[node, :] = transpose(net.output_blobs[:aggregator].data)
        destroy(net)
        shutdown(backend)
    end

    return output_predictions
end

In [40]:
#datainput_train

In [41]:
output_nodes = evaluate_test_data_with_local_models_nodes(4, examples)
output = evaluate_test_data_with_local_models(examples)

09-Oct 20:07:36:INFO:root:Constructing net MLP on Mocha.CPUBackend...
09-Oct 20:07:36:INFO:root:Topological sorting 4 layers...
09-Oct 20:07:36:INFO:root:Setup layers...
09-Oct 20:07:36:INFO:root:Network constructed!
09-Oct 20:07:36:INFO:root:Loading existing model from Snapshots/snapshots_2/snapshot-010000.jld
09-Oct 20:07:36:DEBUG:root:Loading parameters for layer ip
09-Oct 20:07:36:DEBUG:root:Loading parameters for layer ip2
09-Oct 20:07:36:DEBUG:root:Loading parameters for layer aggregator
09-Oct 20:07:36:DEBUG:root:Destroying network MLP
09-Oct 20:07:36:INFO:root:Constructing net MLP on Mocha.CPUBackend...
09-Oct 20:07:36:INFO:root:Topological sorting 4 layers...
09-Oct 20:07:36:INFO:root:Setup layers...
09-Oct 20:07:36:INFO:root:Network constructed!
09-Oct 20:07:36:INFO:root:Loading existing model from Snapshots/snapshots_3/snapshot-010000.jld
09-Oct 20:07:36:DEBUG:root:Loading parameters for layer ip
09-Oct 20:07:36:DEBUG:root:Loading parameters for layer ip2
09-Oct 20:07:36:DEB

1000-element Array{Float64,1}:
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
  ⋮         
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278
 -0.00613278

In [42]:
@everywhere function R2(predictions, y)
    SSres = sum((predictions .- y).^2)
    ymean = mean(y)
    SStot = sum((y .- ymean).^2)
    return 1 - (SSres/SStot)
end

In [43]:
id=string(2)
datainput = load("Data/data_"*id*"/input_test.jld", "dataset_input")
dataoutput = load("Data/data_"*id*"/output_test.jld", "dataset_output")

1000-element Array{Float64,1}:
  0.00600622
 -0.0798315 
 -0.0354346 
  0.0452052 
  0.135951  
 -0.15978   
  0.0114424 
 -0.0460349 
 -0.159059  
 -0.127015  
  0.00895155
  0.127441  
 -0.0411595 
  ⋮         
 -0.0626507 
  0.171655  
  0.0276671 
 -0.0522661 
  0.00694757
  0.0510112 
  0.00466819
 -0.104247  
  0.0142699 
  0.12716   
 -0.121729  
  0.0576027 

In [44]:
output_mean = zeros(examples)
output_mean[:] = mean(datalabels)

-0.0010081986970164

In [45]:
R2(output_mean, dataoutput)

-0.00011044567726337107

In [46]:
R2(output, dataoutput)

-0.0018397673133603298

In [47]:
R2(transpose(output_nodes[1,:]), dataoutput)

-0.0018397673133603298

In [48]:
@everywhere function MAPE(predictions, y)
    return 100*mean(abs((y .- predictions)./y))
end


In [50]:
(output .- dataoutput) ./ output

1000-element Array{Float64,1}:
 -0.012139 
  0.0736987
  0.0293018
 -0.0513379
 -0.142084 
  0.153647 
 -0.0175752
  0.0399021
  0.152926 
  0.120882 
 -0.0150843
 -0.133574 
  0.0350268
  ⋮        
  0.0565179
 -0.177788 
 -0.0337999
  0.0461333
 -0.0130803
 -0.057144 
 -0.010801 
  0.0981145
 -0.0204027
 -0.133293 
  0.115596 
 -0.0637355

# Summary Statistics

In [86]:
summa = [k for k in mean(dataset,1)]
push!(summa, kurtosis(dataset,1))

3-element Array{Any,1}:
  2.02381 
  1.98807 
 -0.520823

In [42]:
@everywhere function order_statistics(dataset, k=5, pow=2)
    powers = 1:pow
    len,attributes = size(dataset)
    orders = [floor(Int,(len/7)*(i-1))+1 for i=1:k]
    statistics = []
    for attribute in 1:attributes
        sorted = sort(dataset[:,attribute])
        for p in powers
            append!(statistics, [sorted[k]^p for k in orders])
        end
    end
    return statistics
end


In [61]:
1./median(abs(dataset .- median(dataset,1)),1)

1x2 Array{Float64,2}:
 1.45699  1.42094

In [64]:
[*[1.,2.]]

LoadError: syntax: "*" is not a unary operator

In [147]:
function SummaryStatistics(X1,X2)
    summary1 = mean(X1,1),var(X1,1),kurtosis(X1,1),skewness(X1,1)
    summary2 = mean(X2,1),var(X2,1),kurtosis(X2,1),skewness(X2,1)
    return summary1, summary2
end

function distance(X,Y,w=0)
    if weights != 0
        d = [((X[i] - Y[i]).*w[i]).^2 for i in 1:length(X)]
    else
        d = [(X[i] - Y[i]).^2 for i in 1:length(X)]
    end
    d = sqrt(sum([[sum(d[i]) for i in 1:length(Y)]...]))
    return d
end

distance (generic function with 2 methods)

In [149]:
X,Y = SummaryStatistics(dataset1, dataset2)
weights = [1,3,3,1]
weights = weights2/sum(weights)
weights
#distance(X,Y,weights)

4-element Array{Float64,1}:
 0.015625
 0.046875
 0.046875
 0.015625

In [120]:
s1 = mean(dataset1,1), var(dataset1,1)
s2 = mean(dataset2,1), var(dataset2,1)
#s2
d = [(s1[i] - s2[i]).^2 for i in 1:length(s1)]
d = sqrt([[sum(d[i]) for i in 1:length(d)]...])
#sqrt(sum((dataset1 .- dataset2).^2))

2-element Array{Float64,1}:
 2.86701  
 0.0764879

In [71]:
kurtosis(dataset[:,1])

0.1882600334061122

In [58]:
dataset[1]

-0.5549318327747421

In [54]:
sum(dataset[1].^2)

0.3079493390267343

In [7]:
using Keras
import Keras.Layers: Dense, Activation

In [8]:
model = Sequential()

LoadError: UndefVarError: Sequential not defined